Based on: https://stackoverflow.com/questions/78801684/how-to-use-2-colors-on-same-line-lets-plot-in-kotlin

Dataset source: https://www.cryptodatadownload.com/data/ (see also data/datasets.txt)

In [1]:
from datetime import timedelta

import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
year = 2024
df = pd.read_csv("data/Bitstamp_BTCUSD_d.csv", parse_dates=["date"])
df = df[df['date'].dt.year == year].sort_values(by='date').reset_index(drop=True)
df = df.assign(**{'bullish': df['close'] > df['open'],
                  'from': df[['open', 'close']].min(axis=1),
                  'to': df[['open', 'close']].max(axis=1),
                  'trend': (df['close'].diff().fillna(0) > 0).astype(int)})
print(df.shape)
df.head()

(366, 13)


,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD,bullish,from,to,trend
0,1704067200,2024-01-01,BTC/USD,42258.0,44192.0,42188.0,44187.0,1633.498618,7.217940e+07,True,42258.0,44187.0,0
1,1704153600,2024-01-02,BTC/USD,44187.0,45922.0,44167.0,44970.0,3948.659076,1.775712e+08,True,44187.0,44970.0,1
2,1704240000,2024-01-03,BTC/USD,44972.0,45510.0,41454.0,42865.0,3980.828056,1.706382e+08,False,42865.0,44972.0,0
3,1704326400,2024-01-04,BTC/USD,42863.0,44795.0,42656.0,44186.0,2918.119349,1.289400e+08,True,42863.0,44186.0,1
4,1704412800,2024-01-05,BTC/USD,44195.0,44366.0,42500.0,44184.0,3155.466383,1.394211e+08,False,44184.0,44195.0,0


In [4]:
def candlestick_chart(data):
    month_name = data['date'].dt.month_name().iloc[0]
    return ggplot(data, aes(x='date')) + \
        geom_boxplot(aes(x='date', ymin='low', ymax='high', lower='from', upper='to', fill='bullish'), stat='identity') + \
        scale_x_datetime('Date', format='%b %e') + \
        scale_fill_manual(name='Bullish', values={True: "#00aa00", False: "#aa0000"}) + \
        labs(title='{0}, {1}'.format(year, month_name), y='Price') + \
        theme_minimal()

gggrid([candlestick_chart(df[df['date'].dt.month == m]) for m in range(1, 13)], ncol=1) + ggtitle("Candlestick Chart (BTC/USD)")

In [5]:
ggplot(df) + \
    geom_line(aes('date', 'close', color='trend'), tooltips='none') + \
    scale_color_manual(name='Trend', values={1: 'red', 0: 'green'}, labels=['Uptrend', 'Downtrend']) + \
    labs(title='Close Price with Trend-Based Coloring', x='Date', y='Close Price') + \
    ggsize(1000, 400) + \
    theme(legend_position='bottom') + \
    ggtb()